In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [30]:
def calculate_win_loss_ratio(matches_df):
   
    teams_dict = {}

    for index, row in matches_df.iterrows():
        home_team = row['home_team_id']
        away_team = row['away_team_id']
        match_result = row['result']

        if home_team not in teams_dict:
            teams_dict[home_team] = {'Win': 0, 'Loss': 0, 'Draw': 0}

        if away_team not in teams_dict:
            teams_dict[away_team] = {'Win': 0, 'Loss': 0, 'Draw': 0}

        if match_result == 'home team win':
            teams_dict[home_team]['Win'] += 1
            teams_dict[away_team]['Loss'] += 1
        elif match_result == 'away team win':
            teams_dict[home_team]['Loss'] += 1
            teams_dict[away_team]['Win'] += 1
        else:
            teams_dict[home_team]['Draw'] += 1
            teams_dict[away_team]['Draw'] += 1

    teams_df = pd.DataFrame.from_dict(teams_dict, orient='index')

    teams_df['Win/Loss Ratio'] = teams_df['Win'] / teams_df['Loss']

    teams_df.reset_index(inplace=True)
    teams_df.columns = ['Team', 'Win', 'Loss', 'Draw', 'Win/Loss Ratio']

    return teams_df[['Team', 'Win/Loss Ratio']]

matches_df = pd.read_csv('silver/matches_new.csv')

result_df = calculate_win_loss_ratio(matches_df)

print(result_df)
result_df.to_csv('win_loss_ratio_g.csv')


    Team  Win/Loss Ratio
0   T-30        1.733333
1   T-46        0.472222
2   T-83        2.040000
3   T-06        1.100000
4   T-87        1.142857
..   ...             ...
83  T-08        0.500000
84  T-76        0.200000
85  T-37        0.000000
86  T-54        0.000000
87  T-59        0.000000

[88 rows x 2 columns]


In [ ]:
def calculate_success_rate(input_file_name):
    
    df = pd.read_csv(input_file_name)

    player_stats = df.groupby('player_id')['converted'].agg(['count', 'sum'])
    player_stats['success_rate'] = player_stats['sum'] / player_stats['count']

    result_df = pd.DataFrame({
        'player_id': player_stats.index,
        'success_rate': player_stats['success_rate']
    })

    result_df.to_csv('penalty_success_rate_stats_g.csv', index=False)

input_csv = 'silver/penalty_kicks_new.csv'  
calculate_success_rate(input_csv)

In [ ]:
def create_appearances_summary_csv(input_file):
   
    df = pd.read_csv(input_file)

    appearances_df = df.groupby('player_id').size().reset_index(name='appearances')

    output_file = 'appearances_summary_g.csv'

    appearances_df.to_csv(output_file, index=False)

input_file = 'silver/player_appearances_new.csv'
create_appearances_summary_csv(input_file)


In [33]:
squads = pd.read_csv('silver/squads_new.csv')
tournament_standings = pd.read_csv('silver/tournament_standings_new.csv')

selected_teams = tournament_standings[['tournament_id', 'team_id']]

final_df = pd.merge(selected_teams, squads, how='inner', on=['tournament_id', 'team_id'])

final_df = final_df[['tournament_id', 'team_id', 'player_id']]

print(final_df[['tournament_id', 'team_id', 'player_id']])

players=pd.read_csv('silver/players_new.csv')
final_df = pd.merge(final_df, players[['player_id','birth_date']], how='inner', on='player_id')

df=final_df

df['tournament_year'] = df['tournament_id'].str.extract(r'(\d+)').astype(int)

df['birth_date'] = pd.to_datetime(df['birth_date'], errors='coerce')

df['age'] = df['tournament_year'] - df['birth_date'].dt.year

df['age'] = df['age'].astype(float)

df.to_csv('age_distribution_histograma_g.csv', index=False)

     tournament_id team_id player_id
0          WC-1930    T-84   P-63826
1          WC-1930    T-84   P-57352
2          WC-1930    T-84   P-63987
3          WC-1930    T-84   P-47453
4          WC-1930    T-84   P-76186
...            ...     ...       ...
1793       WC-2022    T-47   P-05216
1794       WC-2022    T-47   P-80656
1795       WC-2022    T-47   P-00597
1796       WC-2022    T-47   P-91543
1797       WC-2022    T-47   P-86644

[1798 rows x 3 columns]


In [4]:
players_df = pd.read_csv('silver/players_new.csv')
goals_df = pd.read_csv('silver/goals_new.csv')

goals_count_df = goals_df.groupby('player_id').size().reset_index(name='goals_number')

merged_df = pd.merge(players_df, goals_count_df, on='player_id', how='inner')

merged_df['goals_number'] = merged_df['goals_number'].fillna(0)


result_df = merged_df.pivot_table(index='player_id', 
                                  values=['goal_keeper', 'defender', 'midfielder', 'forward', 'goals_number'],
                                  aggfunc={'goal_keeper': 'max', 'defender': 'max', 
                                           'midfielder': 'max', 'forward': 'max', 'goals_number': 'sum'}).reset_index()

result_df[['goal_keeper', 'defender', 'midfielder', 'forward']] = result_df[['goal_keeper', 'defender', 'midfielder', 'forward']].astype(int)

goalkeeper_df = result_df[result_df['goal_keeper'] == 1]
defender_df = result_df[result_df['defender'] == 1]
midfielder_df = result_df[result_df['midfielder'] == 1]
forward_df = result_df[result_df['forward'] == 1]

result_df = pd.DataFrame({
    'position': ['goalkeeper', 'defender', 'midfielder', 'forward'],
    'total_goals': [
        goalkeeper_df['goals_number'].sum(),
        defender_df['goals_number'].sum(),
        midfielder_df['goals_number'].sum(),
        forward_df['goals_number'].sum()
    ]
})

print(result_df)


result_df.to_csv('pie_chart_pjesa1_g.csv', index=False)



     position  total_goals
0  goalkeeper            5
1    defender          437
2  midfielder         1254
3     forward         2427


In [5]:
df = pd.read_csv("silver/group_standings_new.csv")

averages_df = df.groupby('team_id').agg({
    'goals_for': 'mean',
    'points': 'mean'
}).reset_index()

averages_df['goals_avg'] = averages_df['goals_for'].round(2)
averages_df['points_avg'] = averages_df['points'].round(2)

averages_df = averages_df.drop(['goals_for', 'points'], axis=1)

averages_df.to_csv('avg_goals_points_g.csv', index=False)


In [6]:
def calculate_goals_avg(input_file):
    
    df = pd.read_csv(input_file)

    grouped_df = df.groupby('team_id').agg({
        'played': 'sum',
        'goals_for': 'sum'
    }).reset_index()

    grouped_df['goals_avg'] = (grouped_df['goals_for'] / grouped_df['played']).round(2)

    grouped_df = grouped_df[['team_id', 'goals_avg']]

    output_file = 'goals_avg_against_g.csv'

    grouped_df.to_csv(output_file, index=False)

input_file = 'silver/group_standings_new.csv'

calculate_goals_avg(input_file)


In [7]:
csv_file_path = 'silver/group_standings_new.csv'

df = pd.read_csv(csv_file_path)

grouped_df = df.groupby('team_id').agg({'played': 'sum', 'wins': 'sum'}).reset_index()

grouped_df['win_probability'] = grouped_df['wins'] / grouped_df['played']

grouped_df['win_probability'] = grouped_df['win_probability'].round(2)

result_df = grouped_df[['team_id', 'win_probability']]

print(result_df)

result_df.to_csv('win_prob_g.csv', index=False)


   team_id  win_probability
0     T-01             0.25
1     T-02             0.00
2     T-03             0.50
3     T-04             0.26
4     T-05             0.36
..     ...              ...
81    T-84             0.46
82    T-85             0.00
83    T-86             0.57
84    T-87             0.50
85    T-88             0.00

[86 rows x 2 columns]


In [8]:
csv_file_path = 'silver/group_standings_new.csv'

df = pd.read_csv(csv_file_path)

grouped_df = df.groupby('team_id').agg({
    'wins': 'sum',
    'draws': 'sum',
    'losses': 'sum'
}).reset_index()

result_df = grouped_df[['team_id', 'wins', 'draws', 'losses']]

print(result_df)

result_df.to_csv('pie_chart_pjesa2_g.csv', index=False)


   team_id  wins  draws  losses
0     T-01     3      3       6
1     T-02     0      2       1
2     T-03    34     12      22
3     T-04    10      9      20
4     T-05     8      4      10
..     ...   ...    ...     ...
81    T-84    19     12      10
82    T-85     0      4       2
83    T-86    21     11       5
84    T-87    14      7       7
85    T-88     0      0       3

[86 rows x 4 columns]


In [9]:
file_path = 'silver/group_standings_new.csv'  
columns_to_extract = ['tournament_id', 'team_id', 'goals_for', 'goals_against', 'points']
df = pd.read_csv(file_path, usecols=columns_to_extract)

df['year'] = df['tournament_id'].str.extract(r'(\d+)').astype(int)

grouped_df = df.groupby(['tournament_id', 'team_id', 'year']).agg({
    'goals_for': 'sum',
    'goals_against': 'sum',
    'points': 'sum'
}).reset_index()

output_file_path = 'line_chart_g.csv'  
grouped_df.to_csv(output_file_path, index=False)

print(grouped_df)


    tournament_id team_id  year  goals_for  goals_against  points
0         WC-1930    T-03  1930         10              4       6
1         WC-1930    T-06  1930          0              4       0
2         WC-1930    T-07  1930          0              8       0
3         WC-1930    T-09  1930          5              2       2
4         WC-1930    T-13  1930          5              3       4
..            ...     ...   ...        ...            ...     ...
589       WC-2022    T-75  2022          4              3       6
590       WC-2022    T-79  2022          1              1       4
591       WC-2022    T-83  2022          2              1       5
592       WC-2022    T-84  2022          2              2       4
593       WC-2022    T-85  2022          1              6       1

[594 rows x 6 columns]


In [10]:
award_winners_df= pd.read_csv("silver/award_winners_new.csv")
award_winners_df['year'] = award_winners_df['tournament_id'].str.extract(r'(\d{4})')
result = award_winners_df.groupby(['year', 'award_name']).size().reset_index(name='count')
print("Awards won each year")
result.to_csv("awards_each_year_g.csv", index=False)
print(result)

Awards won each year
     year    award_name  count
0    1930   Bronze Boot      1
1    1930   Golden Boot      1
2    1930   Silver Boot      1
3    1934   Golden Boot      1
4    1934   Silver Boot      2
..    ...           ...    ...
167  2022   Golden Ball      1
168  2022   Golden Boot      1
169  2022  Golden Glove      1
170  2022   Silver Ball      1
171  2022   Silver Boot      1

[172 rows x 3 columns]


In [11]:
award_winners = pd.read_csv('silver/award_winners_new.csv')
squads = pd.read_csv('silver/squads_new.csv')

merged_data = pd.merge(award_winners, squads, on='player_id', how='inner')

position_award_data = merged_data[['position_name', 'award_name']]

position_award_data.to_csv('positionaward_3a_g.csv', index=False)

print(position_award_data)

    position_name         award_name
0         forward        Golden Boot
1         forward        Silver Boot
2         forward        Bronze Boot
3         forward        Golden Boot
4         forward        Golden Boot
..            ...                ...
550       forward        Bronze Boot
551       forward        Bronze Boot
552       forward        Bronze Boot
553   goal keeper       Golden Glove
554    midfielder  Best Young Player

[555 rows x 2 columns]


In [12]:
award_winners = pd.read_csv('silver/award_winners_new.csv')

award_winners['year'] = award_winners['tournament_id'].str.extract(r'(\d{4})', expand=False)

shared_award_winners = award_winners[award_winners['shared'] == 1]

shared_awards_count = shared_award_winners.groupby('year').size().reset_index(name='count')

shared_awards_count.to_csv('sharedawards_count_3b_g.csv', index=False)

print(shared_awards_count)

    year  count
0   1934      2
1   1938      3
2   1950      4
3   1954      3
4   1958      2
5   1962      6
6   1966      4
7   1974      2
8   1986      3
9   1990      2
10  1991      2
11  1994      4
12  1998      2
13  1999      4
14  2002      2


In [16]:
award_winners = pd.read_csv('silver/award_winners_new.csv')

award_winners['shared'] = pd.to_numeric(award_winners['shared'], errors='coerce')

awards_by_nationality = award_winners.groupby('team_name')['shared'].sum().reset_index(name='total_awards_won')

print(awards_by_nationality)

         team_name  total_awards_won
0        Argentina                 2
1        Australia                 0
2          Austria                 1
3          Belgium                 0
4           Brazil                 8
5         Bulgaria                 1
6         Cameroon                 1
7           Canada                 0
8            Chile                 1
9            China                 2
10        Colombia                 0
11         Croatia                 0
12  Czechoslovakia                 0
13         Denmark                 0
14         England                 2
15          France                 0
16         Germany                 2
17         Hungary                 4
18           Italy                 3
19           Japan                 0
20     Netherlands                 1
21          Norway                 1
22            Peru                 0
23          Poland                 1
24        Portugal                 0
25          Russia                 1
2